In [2]:
import pandas as pd

In [3]:
dados = pd.read_csv("vgsales.csv")

In [4]:
#remove valores NaN
dados = dados.dropna()

In [5]:
#obtendo apenas variaveis desejadas
#desjamos prever a venda de um videogame na america do norte em função da plataforma, ano, genero e autora
dataset = dados.loc[:, ["Platform", "Year", "Genre" , "Publisher", "NA_Sales"]]

In [6]:
#escolhendo consoles para comparar
consoles = dataset["Platform"].value_counts()

In [7]:
#plataformas relevantes
relevantes = (dataset["Platform"] == "PS3") | (dataset["Platform"] == "Wii") |  (dataset["Platform"] == "X360")
dataset = dataset.loc[relevantes,:]

In [25]:
#Peganado top 10 publishers
top_10 = dataset["Publisher"].value_counts().head(10)
top_10_dict = top_10.to_dict()
chav = top_10_dict.keys()


keys = []


for i in chav:
    keys.append(i)
top_10

Electronic Arts                 395
Activision                      350
Ubisoft                         285
Namco Bandai Games              214
THQ                             185
Konami Digital Entertainment    162
Take-Two Interactive            161
Sega                            148
Sony Computer Entertainment     128
Capcom                           95
Name: Publisher, dtype: int64

In [28]:
#filtrando publishers
filtro = (dataset["Publisher"] == keys[0]) | (dataset["Publisher"] == keys[1]) | \
(dataset["Publisher"] == keys[2]) | (dataset["Publisher"] == keys[3]) | (dataset["Publisher"] == keys[4])\
| (dataset["Publisher"] == keys[5]) | (dataset["Publisher"] == keys[6]) | (dataset["Publisher"] == keys[7])\
| (dataset["Publisher"] == keys[8]) | (dataset["Publisher"] == keys[9])

game = dataset.loc[filtro,:]

In [31]:
#DataSet com Dummies
#
##
###
#####
######
#######
gamed = pd.get_dummies(game, columns = ["Platform", "Genre", "Publisher"])
#######
######
#####
###
##
#

In [121]:
PS3 = game["Platform"] == "PS3"
Wii = game["Platform"] == "Wii"
X360 = game["Platform"] == "X360"

PS3 = game.loc[PS3,:]
Wii = game.loc[Wii,:]
X360 = game.loc[X360,:]

310.97

In [122]:
PS3_sales = PS3["NA_Sales"]
PS3_sales.sum()

310.97

In [123]:
X360_sales = X360["NA_Sales"]
X360_sales.sum()

386.04999999999995

In [124]:
Wii_sales = Wii["NA_Sales"]
Wii_sales.sum()

203.14